# Segmenting and Clustering Neighborhoods in Toronto

## Question 1 - Scraping Wikipedia

In [1]:
import pandas as pd
print('Pandas library imported.')
import requests
print('Requests library imported.')
from bs4 import BeautifulSoup
print('BeautifulSoup library imported.')

print('All libraries imported.')

Pandas library imported.
Requests library imported.
BeautifulSoup library imported.
All libraries imported.


**Scrape the Wikipedia page and transform it into a dataframe with BeautifulSoup and Pandas**

In [2]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(result.content,'lxml')
table = soup.find_all('table')[0] 
neighborhoods = pd.read_html(str(table))[0]

**The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**

In [3]:
# Rename columns according to the assignment
neighborhoods.columns = ['PostalCode', 'Borough', 'Neighborhood']

**Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned**

In [4]:
# Remove lines for which column Borough has value "Not assigned"
neighborhoods = neighborhoods[neighborhoods['Borough']!='Not assigned']

**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table**

In [5]:
# The Wikipedia data is already grouped by postal code (update since the creation of the assignement?), we just have to replace " /" by ","
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace(' /', ',')

**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough**

In [6]:
neighborhoods.loc[neighborhoods['Neighborhood'] =='Not assigned' , 'Neighborhood'] = neighborhoods['Borough']

**Cleaning**

In [7]:
# Reset index
neighborhoods = neighborhoods.reset_index()
neighborhoods.drop(['index'], axis='columns', inplace=True)

**In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe**

In [8]:
# Print the shape
print('The dataframe has the following shape: {}.'.format(
        neighborhoods.shape,
    )
)

The dataframe has the following shape: (103, 3).


## Question 2 - Get latitude and logitude of each postal code

In [ ]:
#!conda install -c conda-forge geocoder
import geocoder
print('Geocoder library imported.')

print('All libraries imported.')

Solving environment: \ 

**Create a function that will loop until coordinates are obtained from Geocoder**

In [ ]:
def get_coordinates_from_geocoder(address):
    # initialize your variable to None
    coordinates = None

    # loop until you get the coordinates
    index = 1
    while(coordinates is None):
        print('Iteration {}'.format(index))
        g = geocoder.google(address)
        coordinates = g.latlng
        index = index + 1
        
        if index == 25:
            break

        return coordinates

**Add 2 new columns to the dataframe**

In [ ]:
neighborhoods['Latitude'] = 0
neighborhoods['Longitude'] = 0

**Use the function created above to create a new data frame with coordinates**

In [ ]:
for index in range(len(neighborhoods)):
    address = '{}, Toronto, Ontario'.format(neighborhoods.loc[index, 'PostalCode'])
    print('-----')
    print('row {} beginning: {}'.format(index, address))
    coordinates = get_coordinates_from_geocoder(address)
    if coordinates is not None:
        neighborhoods.loc[index, 'Latitude'] = coordinates[0]
        neighborhoods.loc[index, 'Longitude'] = coordinates[1]
    else:
        print("Error with Google Geocoding")
    print('{}%: row {} completed'.format(round(index/len(neighborhoods.index)*100, 1), index))

**Because Google Geocoding is not working, we are using the provided csv**

In [ ]:
# Download the csv file
!wget -q -O 'Toronto_coordinates.csv' http://cocl.us/Geospatial_data
toronto_coordinates = pd.read_csv('Toronto_coordinates.csv')
toronto_coordinates.head()

**Merge the 2 dataframes**

In [ ]:
# Prepare the coordinates dataframe to be merged by changing the column names so they correspond to the neighborhood dataframe
toronto_coordinates.columns=['PostalCode','Latitude','Longitude']

# Remove the 2 columns created during the attempt to make Google Geocoding work to avoid duplicates
neighborhoods.drop(['Latitude'], axis='columns', inplace=True)
neighborhoods.drop(['Longitude'], axis='columns', inplace=True)

# Merge
neighborhoods = pd.merge(neighborhoods, toronto_coordinates[['PostalCode','Latitude', 'Longitude']], on='PostalCode')

**Display the dataframe**

In [ ]:
neighborhoods

# Question 3 - Cluster the neighborhoods in Toronto 

**Import libraries**

In [ ]:
import numpy as np # library to handle data in a vectorized manner
print('Numpy library imported.')

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Geopy library imported.')

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Matplotlib libraries imported.')

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('KMeans library imported.')

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Folium library imported.')

print('Libraries imported.')

**Use geopy library to get the latitude and longitude values of Toronto**

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

**Create a map of Toronto with neighborhoods superimposed on top**

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto